# All useful python code I've used for the project

### Format of the dataset

Only mentioning parts used in this code

```
<listOfUnitDefinitions>
<listOfCompartments>
<listOfSpecies>
    <species metaid="" id="" name="" compartment="" etc...> #metaid = id 
<listOfParameters>
    <parameter sboTerm="" id="" value=""> # 3 possible ids: FB1N1000, FB2N0, FB3N1000 (equivalent to -1000, 0 and 1000)
<listOfReactions>
    <reaction metaid="" id="" name="" reversible="" fbc:lowerFluxBound="" fbc:upperFluxBound> #metaid = id = name
        <listOfReactants>
            <speciesReference species="" stoichiometry="" etc...>
        <listOfProducts>
            <speciesReference species="" stoichiometry="" etc...>
<fbc:listOfObjectives>
    <fbc:objective fbc:id="" fbc:type=""> #type = maximise or minimise
        <fbc:listOfFluxObjectives>
            <fbc:fluxObjective fbc:reaction="" fbc:coefficient=""> #fbc:reaction=objective function reaction's ID
<fbc:listOfGeneProducts>
<groups:listOfGroups>
    <groups:group sboTerm="" groups:id="" groups:name="" groups:kind="">
        <groups:listOfMembers>
            <groups:member groups:idRef=""> #individual reaction IDs
```

### Output format required (as JSON)

```
{
    "nodes": [
        {
            "id": "",
            "name": "",
            "group": ""
        },
        {
            "id": "",
            "name": "",
            "group": ""
        }
     ],
     "links": [
         {
             "source": "",
             "target": ""
         },
         {
             "source": "",
             "target": ""
         }
     ]
}
```
Additional keys can be added, e.g. color, particle, etc.

### Setup

In [2]:
import xml.dom.minidom
import cobra
import pandas as pd
import numpy as np
import json

### Main dataframe for connecting species names, groups and IDs

In [8]:
file = xml.dom.minidom.parse("icardio.xml") #read xml file
model = file.documentElement #take all elements in the xml as the model
species = model.getElementsByTagName("species") #search model for all elements called "species" - refers to chemical species

species_names = []
species_groups = []
species_ids = []

for element in species: #loop through elements and append name, compartment and ID attributes to appropriate lists
    species_names.append(element.getAttribute("name"))
    species_groups.append(element.getAttribute("compartment"))
    species_ids.append(element.getAttribute("id"))
     
dataframe = pd.DataFrame({"names": species_names, "groups": species_groups, "ids": species_ids}) #make dataframe from lists
dataframe.index = np.arange(1, len(dataframe)+1)
%store dataframe 
#I stored this dataframe to use in other jupyter notebooks
dataframe #format visible below after running

Stored 'dataframe' (DataFrame)


,names,groups,ids
1,(10Z)-heptadecenoic acid,c,m00003__91__c__93__
2,(10Z)-heptadecenoic acid,r,m00003__91__r__93__
3,(10Z)-heptadecenoic acid,s,m00003__91__s__93__
4,(10Z)-heptadecenoyl-CoA,c,m00004__91__c__93__
5,(10Z)-heptadecenoyl-CoA,m,m00004__91__m__93__
...,...,...,...
2886,hyocholoyl-CoA,c,m90185__91__c__93__
2887,protein,c,m90190__91__c__93__
2888,glycogen storage pool,c,m90194__91__c__93__
2889,intracellular lipids,c,m90195__91__c__93__


### Dataframe for organising and connecting reactions to their reactants and products

The code below loops through the "reaction" elements in the XML file. It is necessary to loop through the list of reactants and list of products separately because the individual species are listed as "speciesReference" under both of the categories. Searching for "speciesReference" would not specify if a species is reactant or product in a reaction.

In [9]:
list_of_reactions = []
reactions = model.getElementsByTagName("reaction") #get all elements in the model that are called "reaction"

for reaction in reactions:
    this_reaction = {}
    this_reaction["id"] = reaction.getAttribute("name") #make the reaction's "name" attribute its ID number in the dataframe
    list_of_reactants = reaction.getElementsByTagName("listOfReactants") #get all elements in the current reaction that are called "listOfReactants"
    list_of_products = reaction.getElementsByTagName("listOfProducts") #get all elements in the current reaction that are called "listOfProducts"
    for reactant in list_of_reactants:
        this_reaction["Reactants"] = []
        species_reference = reactant.getElementsByTagName("speciesReference") #get all elements in the current reactant element that are called "speciesReference"
        for species in species_reference: 
            this_reaction["Reactants"].append({"id": species.getAttribute("species"), #organise into list of dictionaries
                                               "name": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "names"].array[0],
                                               "group": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "groups"].array[0]})
    for product in list_of_products:
        this_reaction["Products"] = []
        species_reference = product.getElementsByTagName("speciesReference") #get all elements in the current product element that are called "speciesReference"
        for species in species_reference:
            this_reaction["Products"].append({"id": species.getAttribute("species"), #organise into list of dictionaries
                                               "name": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "names"].array[0],
                                               "group": dataframe.loc[dataframe["ids"] == species.getAttribute("species"), "groups"].array[0]})        
        
    list_of_reactions.append(this_reaction)

%store list_of_reactions 
#storing list_of_reactions for use in other jupyter notebooks
list_of_reactions[0] #output format visible below after running

Stored 'list_of_reactions' (list)


{'id': 'RCR10001',
 'Reactants': [{'id': 'm01285__91__c__93__', 'name': 'ADP', 'group': 'c'}],
 'Products': [{'id': 'm01334__91__c__93__', 'name': 'AMP', 'group': 'c'},
  {'id': 'm01371__91__c__93__', 'name': 'ATP', 'group': 'c'}]}

### Dataframe for associating reactions and their reactants/products with metabolic pathways

In [10]:
pathways = model.getElementsByTagName("groups:group") #get all elements in the model that are called "groups:group", the groups are metabolic pathways
list_of_pathways = []
for item in pathways:
    if item.hasAttribute("groups:name"): #get the pathway's name into a list
        name = item.getAttribute("groups:name")
    list_of_members = item.getElementsByTagName("groups:listOfMembers") #get the list of reactions participating in the current metabolic pathway, this is necessary because the actual items I was interested in are nested inside
    for member in list_of_members:
        member_rxn = member.getElementsByTagName("groups:member") #get individual reactions from the list
        name_list = []
        for rxn in member_rxn: #loop through each reaction
            rxn_item = {}
            rxn_item["id"] = rxn.getAttribute("groups:idRef") #get all attributes that are called "groups:idRef" from the current reaction
            for item in list_of_reactions: #loop through list created in the above cell
                if item["id"] == rxn_item["id"]: #if the reaction ID is found, take the stored dictionaries and append them to the pathway list
                    rxn_item["Reactants"] = item["Reactants"]
                    if "Products" in item:
                        rxn_item["Products"] = item["Products"]
            name_list.append(rxn_item)
        pathway_dict = {}
        pathway_dict["name"] = name
        pathway_dict["Reactions"] = name_list
        list_of_pathways.append(pathway_dict) #output format visible in below cell

In [ ]:
fatty_acid = list_of_pathways[15] 
%store fatty_acid 
#storing fatty acid metabolism reactions for use in other jupyter notebooks
fatty_acid

### Further filtering and organising

Since the data is mostly in the right format now it can be accessed and appended into other lists.

#### Optimising for biomass/ATP

The model is optimised for biomass by default. To optimise for ATP, I've chosen reaction "RCR20085": ADP + Pi + H+ → ATP + H+ + H2O

In [11]:
ATP_optimisation = cobra.io.read_sbml_model("icardio.xml") #read in model
ATP_optimisation.objective = "RCR20085" #change objective function, this step is omitted when optimising for biomass
ATP_optimisation.optimize() #optimise to calculate a possible solution
ATP_optimisation_rxn_names = [_.name for _ in ATP_optimisation.reactions] #create list of reaction names
ATP_optimisation_rxn_flux = [round(_.flux, 3) for _ in ATP_optimisation.reactions] #create list of reaction fluxes rounded to 3 decimal places

#### Optional filtering: glucose levels

To reduce or completely cut off glucose the flux bounds of the appropriate reactions need to be set to the reduced percentage or 0. In the below example I wanted to optimise for ATP and cut off glucose completely. I looked up the appropriate reactions (just checked manually in the xml) and changed the lower and upper bounds to 0. The reactions are the transport reactions of glucose from cytoplasm to sarcoplasmic reticulum (glc_c_to_s), sarcoplasmic reticulum to cytoplasm (glc_s_to_c), cytoplasm to golgi (glc_c_to_g), and cytoplasm to ribosome (glc_c_to_r).

I was also interested in what happens at 25% glucose. For this I changed the lower bound to -250 and the upper bound to 250. This equates to 25% in either direction.

In [ ]:
ATP_no_glc = cobra.io.read_sbml_model("icardio.xml")
ATP_no_glc.objective = "RCR20085"
glc_c_to_s = ATP_no_glc.reactions.get_by_id("RCR41035")
glc_s_to_c = ATP_no_glc.reactions.get_by_id("RCR40464")
glc_c_to_g = ATP_no_glc.reactions.get_by_id("RCR20130")
glc_c_to_r = ATP_no_glc.reactions.get_by_id("RCR20071")
glc_c_to_s.lower_bound, glc_c_to_s.upper_bound = 0, 0 #-250, 250 for 25% glucose
glc_s_to_c.lower_bound, glc_c_to_s.upper_bound = 0, 0 #-250, 250 for 25% glucose
glc_c_to_g.lower_bound, glc_c_to_s.upper_bound = 0, 0 #-250, 250 for 25% glucose
glc_c_to_r.lower_bound, glc_c_to_s.upper_bound = 0, 0 #-250, 250 for 25% glucose
ATP_no_glc.optimize()
no_glc_rxn_names = [_.name for _ in ATP_no_glc.reactions]
no_glc_rxn_flux = [round(_.flux, 3) for _ in ATP_no_glc.reactions]

#### Optional filtering: flux difference

I wanted to show the differences in flux between two conditions, for example between biomass optimised and ATP optimised. 

In [ ]:
biomass_ATP_flux_difference = []
for index, item in enumerate(biomass_rxn_flux): #requires running the optimisation cell before with an appropriate variable for the fluxes of both conditions
    biomass_ATP_flux_difference.append(item - ATP_rxn_flux[index]) 

#### Optional filtering: compartments

In some cases the dataset was so large that it really slowed the graphing process down so I opted to show certain compartments only. The main organising script just needs to check whether the reaction is within the selected reactions. 

In [ ]:
metabolism = list_of_reactions #fatty_acid["Reactions"] can be swapped for list_of_reactions if we're working with whole-cell metabolism
selector = "c" #c for cytoplasm for example
selected_reactions = []

for item in metabolism: 
    for reactant in item["Reactants"]:
        if reactant["group"] == selector and item["id"] not in selected_reactions:
            selected_reactions.append(item["id"])
    if "Products" in item:
        for product in item["Products"]:
            if product["group"] == selector and item["id"] not in selected_reactions:
                selected_reactions.append(item["id"])

#### Separating non-zero flux values

This is necessary because zero flux reactions are not happening and need to be separated.

In [12]:
non_zero = []
positive = []
negative = []
zero = []

for index, flux in enumerate(ATP_optimisation_rxn_flux):
    if flux > 0:
        non_zero.append(ATP_optimisation_rxn_names[index])
        positive.append(ATP_optimisation_rxn_names[index])
    elif flux < 0:
        non_zero.append(ATP_optimisation_rxn_names[index])
        negative.append(ATP_optimisation_rxn_names[index])
    else:
        zero.append(ATP_optimisation_rxn_names[index])

#### Colours

I associated increasing ranges of flux with increasingly green colour. I take the absolute values of the fluxes because only the magnitude matters, not the direction.

In [15]:
flux_colours = []

for item in ATP_optimisation_rxn_flux:
    if item == 0:
        flux_colours.append("rgba(255, 255, 255, 0.1)") #white
    elif 0 < abs(item) <= 200:
        flux_colours.append("rgba(213, 232, 231, 1)")
    elif 200 < abs(item) <= 400:
        flux_colours.append("rgba(171, 208, 207, 1)")
    elif 400 < abs(item) <= 600:
        flux_colours.append("rgba(129, 185, 183, 1)")
    elif 600 < abs(item) <= 800:
        flux_colours.append("rgba(82, 163, 160, 1)")
    elif 800 < abs(item) <= 1000:
        flux_colours.append("rgba(0, 140, 138, 1)") #full green/teal
        
flux_dataframe = pd.DataFrame({"name": ATP_optimisation_rxn_names, "flux": ATP_optimisation_rxn_flux, "colour": flux_colours})
flux_dataframe

,name,flux,colour
0,RCR10001,0.0,"rgba(255, 255, 255, 0.1)"
1,RCR10002,0.0,"rgba(255, 255, 255, 0.1)"
2,RCR10004,-500.0,"rgba(129, 185, 183, 1)"
3,RCR10005,0.0,"rgba(255, 255, 255, 0.1)"
4,RCR10006,0.0,"rgba(255, 255, 255, 0.1)"
...,...,...,...
4195,RCR90207,0.0,"rgba(255, 255, 255, 0.1)"
4196,RCR90208,0.0,"rgba(255, 255, 255, 0.1)"
4197,RCR90209,0.0,"rgba(255, 255, 255, 0.1)"
4198,RCR90210,0.0,"rgba(255, 255, 255, 0.1)"


#### Making nodes and links

This is the main filtering/organising script. It separates every reaction and compound based on flux and direction while avoiding node duplication and also assigning appropriate colour, particle, flux and name values to each node and link. It also works to create data for the two types of graphs: non-zero flux reactions showing only or non-zero flux reactions highlighted in front of the 0 flux reactions.

In [16]:
metabolism = list_of_reactions #list_of_reactions can be swapped for fatty_acid["Reactions"] if we're only interested in fatty acid metabolism reactions, since the data structure is the same the rest of the code will still work
non_zero_nodes = []
non_zero_links = []
non_zero_pseudo_s = [] #pseudo nodes are required for reactions with multiple reactants/products
non_zero_pseudo_e = [] #if there were no pseudo nodes, all compounds would point at each other and it would make no sense. They are invisible in the final graph
node_counter = []
non_zero_counter = []

for item in metabolism: 
    if item["id"] in non_zero: #check if reaction has non-zero flux, can also add condition here to check if the reaction ID is within the selected_reactions list if filering by compartment
        for reactant in item["Reactants"]:
            if reactant["id"] not in node_counter: #I needed this because I wanted to avoid having multiple nodes that are the same compound
                reactant["opacity"] = False #adding an opacity value to be able to filter showing these nodes in the graphing library
                non_zero_nodes.append(reactant)
                node_counter.append(reactant["id"])
                non_zero_counter.append(reactant["id"]) #I added this because later code would override non-zero values and I'd end up omitting reactions
        if "Products" in item: #some reactions do not have products in the dataset
            non_zero_pseudo_s.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"}) #pseudo nodes get a special group "z" tag so that I can filter them in the final graph
            non_zero_pseudo_e.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
            for reactant in item["Reactants"]:
                if item["id"] in positive: #separating by positive and negative fluxes because this decides the direction of the reaction and the movement of the particle. Also adding links between pseudo nodes where and name, particle, flux, and colour values so that they can be displayed in the graph
                    non_zero_links.append({"source": reactant["id"], "target": item["id"] + "s", "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
                    non_zero_links.append({"source": item["id"] + "s", "target": item["id"] + "e", "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
                elif item["id"] in negative:
                    non_zero_links.append({"source": item["id"] + "s", "target": reactant["id"], "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
                    non_zero_links.append({"source": item["id"] + "e", "target": item["id"] + "s", "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
            for product in item["Products"]:
                if item["id"] in positive:
                    non_zero_links.append({"source": item["id"] + "e", "target": product["id"], "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
                elif item["id"] in negative:
                    non_zero_links.append({"source": product["id"], "target": item["id"] + "e", "name": item["id"], "particle": True, "flux": abs(flux_dataframe.loc[flux_dataframe["name"] == item["id"], "flux"].array[0]), "colour": flux_dataframe.loc[flux_dataframe["name"] == item["id"], "colour"].array[0]})
                if product["id"] not in node_counter: #again avoiding duplication of compound nodes
                    product["opacity"] = False
                    non_zero_nodes.append(product)
                    node_counter.append(product["id"])
                    non_zero_counter.append(product["id"])

#this bit below is only required for the maps where the 0 flux reactions are showing as transparent grey connections while the non-zero flux reactions are highlighted
zero_nodes = []
zero_links = []
pseudo_s = []
pseudo_e = []
pseudo_links = []
node_counter_2 = []

for item in metabolism: #again list_of_reactions needs to be swapped for fatty_acid["Reactions"] when working with fatty acid metabolism reactions only
    if item["id"] in zero:
        for reactant in item["Reactants"]:
            if reactant["id"] not in node_counter_2 and reactant["id"] not in non_zero_counter: #avoiding node duplication
                reactant["opacity"] = True
                zero_nodes.append(reactant)
                node_counter_2.append(reactant["id"])
        if "Products" in item:
            pseudo_s.append({"id": item["id"] + "s", "name": item["id"] + "s", "group": "z"})
            pseudo_e.append({"id": item["id"] + "e", "name": item["id"] + "e", "group": "z"})
            for reactant in item["Reactants"]: #I just assigned a white colour with 0.1 opacity/alpha to all reactions
                zero_links.append({"source": reactant["id"], "target": item["id"] + "s", "colour": "rgba(255, 255, 255, 0.1)"})
            for product in item["Products"]:
                zero_links.append({"source": item["id"] + "e", "target": product["id"], "colour": "rgba(255, 255, 255, 0.1)"})
                if product["id"] not in node_counter_2 and product["id"] not in non_zero_counter:
                    product["opacity"] = True
                    zero_nodes.append(product)
                    node_counter_2.append(product["id"])

                    
for item in pseudo_s: #making a connection between pseudo nodes
        pseudo_links.append({"source": item["id"], "target": item["id"].replace("s", "e"), "colour": "rgba(255, 255, 255, 0.1)"})


#combine compound nodes and pseudo nodes        
all_nodes = non_zero_nodes + non_zero_pseudo_s + non_zero_pseudo_e #+ zero_nodes + pseudo_s + pseudo_e <-- this is only required for the highlighted version
#all_links = non_zero_links + zero_links + pseudo_links <-- this is only required for the highlighted version

#### Making up the correct format

In [ ]:
JSON_export = {"nodes": [], "links": []}
JSON_export["nodes"] += all_nodes
JSON_export["links"] += non_zero_links #or all_links depending on whether it's the non-zero flux only or the highlighted version

### JSON export

In [ ]:
json_string = json.dumps(JSON_export)

with open('appropriate_filename.json', 'w', encoding='utf-8') as f:
    json.dump(JSON_export, f, ensure_ascii=False, indent=4)